In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
# os.chdir(os.path.join(base_dir, 'serverproject'))
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData

from core import services
from core import utils
from core import controllers

In [3]:
out=await utils.generate_image(" ")

In [5]:
# download the image ['https://replicate.delivery/yhqm/2eJWr0h5mUwsUKRw9OB10jzhRhx1fQlWveCAKclhV7dws8omA/out-0.webp']
# download from url as base64
import base64
import requests

def download_image(url):
    response = requests.get(url)
    return base64.b64encode(response.content).decode('utf-8')


img_b64=download_image('https://replicate.delivery/yhqm/2eJWr0h5mUwsUKRw9OB10jzhRhx1fQlWveCAKclhV7dws8omA/out-0.webp')

In [9]:
print(len(img_b64))

159664


In [8]:
# convert base64 to jpg and save

import io
import PIL.Image

def save_image_from_base64(img_b64, filename):
    img_data = base64.b64decode(img_b64)
    img = PIL.Image.open(io.BytesIO(img_data))
    img.save(filename, 'JPEG')

save_image_from_base64(img_b64, 'test.jpg')


In [4]:
out

['https://replicate.delivery/yhqm/2eJWr0h5mUwsUKRw9OB10jzhRhx1fQlWveCAKclhV7dws8omA/out-0.webp']

In [ ]:
limited_recap_data=await utils.get_all_recaps_fast()

In [12]:
import datetime

In [ ]:
# end_of_consideration_date
end_of_consideration_date=datetime.datetime(year=2024, month=8, day=10)
earliest_stream_in_zone=end_of_consideration_date-datetime.timedelta(days=7)

In [ ]:
stream_date_dict={}
consideration_ids=[]
for recap in limited_recap_data:
    # Get the stream date
    stream_recap_limited=StreamRecapData(**recap)
    stream_date=stream_recap_limited.video_characteristics.get('date',"")
    if stream_date=="":
        title=stream_recap_limited.video_characteristics.get("title","")
        stream_date=title.split("Stream Date~")[-1].strip()
    
    #use datetime to check if the date is valid
    try:
        stream_date=datetime.datetime.strptime(stream_date, "%m/%d/%Y")
    except ValueError:
        continue
    
    # Check if the stream is within the week
    if stream_date>=earliest_stream_in_zone and stream_date<=end_of_consideration_date:
        consideration_ids.append(stream_recap_limited.video_id)
        stream_date_dict[stream_recap_limited.video_id]=stream_date


In [ ]:
def get_stream_rundown(full_recap):
    abstractions=full_recap.plot_object["abstractions"]
    stream_rundown="Stream Title: "+full_recap.video_characteristics["title"]+"\nStream Date: "+stream_date_dict[full_recap.video_id].strftime("%A")+"\nTOPICS PROPORTION AND CONTENT:\n"
    abstraction_rundowns=""
    for key, value in abstractions.items():
        if key!="non categorized":
            abstraction_rundown="   Topic: "+key+" | Relative Proportion: "+str(int(value["size"]*100))+"%\nCONTENT: "+value["recap"]
            abstraction_rundowns+=abstraction_rundown+"\n\n"

    stream_rundown+=abstraction_rundowns
    return stream_rundown

In [ ]:
full_stream_recap_datas=[]
stream_rundowns="Today is "+str(datetime.datetime.now().strftime("%A"))+"\nStream Rundowns Start Below:\n\n"
for id in consideration_ids[::-1]:
    try:
        stream_recap_data=await utils.get_recap_data(id)
        stream_rundowns+=get_stream_rundown(stream_recap_data)+"\n\n"
        full_stream_recap_datas.append(stream_recap_data)
    except:
        continue

In [ ]:
print(stream_rundowns)

In [ ]:
system_prompt="""The user will give you a stream rundown for all the streams of the last week and your job is to make a week recap.

Don't talk about percentages of topics or the exact date, use the day of the week when referring to the stream date."""

user_prompt="Here are all the stream rundowns:\n"+stream_rundowns

prompt=[{"role":"system", "content": system_prompt}, {"role":"user", "content": stream_rundowns}]

week_recap, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [ ]:
print(week_recap)

In [ ]:
system_prompt="""The user will give you a stream rundown for all the streams of the last week and your job is to make a week recap.

Don't talk about percentages of topics or the exact date, use the day of the week when referring to the stream date.

Your recap should be a 1 paragraph 3-5 sentence summary of what happened with as much specifics as possible. Be informal to condense the content as it doesn't need to be gramatically correct. SPECIFICS ARE KEY."""

user_prompt="Here are all the stream rundowns:\n"+stream_rundowns

prompt=[{"role":"system", "content": system_prompt}, {"role":"user", "content": stream_rundowns}]

week_recap, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [ ]:
print(week_recap)

In [ ]:
system_prompt="""The user will give you a stream rundown for all the streams of the last week and your job is to make a prompt for an image generator to symbolize the week."""

user_prompt="Here are all the stream rundowns:\n"+stream_rundowns

prompt=[{"role":"system", "content": system_prompt}, {"role":"user", "content": stream_rundowns}]

week_recap, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [ ]:
print(week_recap)

In [ ]:
system_prompt="""The user will give you a stream rundown for all the streams of the last week and your job is to make a headline hook for the week. It should be more like topics separated by commas and not a full sentence.

Do not say anything other than the hook and get right into it. It should be max 3 items."""

user_prompt="Here are all the stream rundowns:\n"+stream_rundowns

prompt=[{"role":"system", "content": system_prompt}, {"role":"user", "content": stream_rundowns}]

week_hook, temp_cost=await utils.async_response_handler(
    prompt,
    utils.ModelNameEnum.claude_3_5_sonnet
)

In [34]:
applied_date=datetime.datetime(2024, 8, 10)

In [39]:
limited_recap_data=await utils.get_all_recaps_fast()
stream_rundowns, weeks_video_ids=await services.timeline_plot.weekly_recaps.get_week_context_prompt(limited_recap_data, applied_date)

cost=0
week_recap, temp_cost=await services.timeline_plot.weekly_recaps.generate_week_recap(stream_rundowns)
cost+=temp_cost

week_hook, temp_cost=await services.timeline_plot.weekly_recaps.generate_week_hook(stream_rundowns)
cost+=temp_cost

In [44]:
week_image, temp_cost=await services.timeline_plot.weekly_recaps.generate_week_image(stream_rundowns)
cost+=temp_cost

In [16]:
image_url='https://replicate.delivery/yhqm/XE6QgEPeZtTHECLmreEH6wmuqO19K4eOwJTuPVfUwIVYS7RNB/out-0.webp'



In [17]:
response = requests.get(image_url)
img_b64=base64.b64encode(response.content).decode('utf-8')


# downscale the iamge to be 400x400
img = PIL.Image.open(io.BytesIO(response.content))

AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

In [18]:
img = img.resize((400, 400))
# make the image a base64 string 
buffered = io.BytesIO()
img.save(buffered, format="JPEG")
img_b64 = base64.b64encode(buffered.getvalue()).decode('utf-8')

In [19]:
# save the image
save_image_from_base64(img_b64, 'week_image.jpg')


In [42]:
image_url=await utils.generate_image("donald_trump")

In [43]:
print(image_url[0])

https://replicate.delivery/yhqm/c3N4yo9n1U7qBVpeYcLLsmMfsODfbJjs1VPfSQb34U5n47RNB/out-0.webp


In [ ]:
# save week image
save_image_from_base64(img_b64, 'week_image.jpg')